## Извлечение данных из веб-страниц

Модуль requests позволяет получать доступ к веб-страницам. 

Запрос типа get - это когда вы передаете серверу какую-то информацию в адресной строке. Например, если вы перейдете по такому адресу: [https://www.google.ru/?q=python+анализ+данных](https://www.google.ru/?q=python+анализ+данных), то этим вы просите гугл искать по запросу "python анализ данных". 

post-запрос - это когда вам нужно ввести информацию в какую-нибудь форму, например, ввести логин-пароль, который не будет отображать в адресной строке браузера.


In [0]:
import requests

---
В качестве примера будем использовать данные о биржевых опционах компании Apple Inc., доступных на сайте Yahoo!Finance  

---

In [0]:
req = requests.get('https://finance.yahoo.com/quote/AAPL/options/')

---
Чтобы проверить, что страница нормально загрузилась используется метод **ok**

---

In [0]:
req.ok 

True

In [0]:
print(req.text)

### Beautiful Soup


Для обработки веб-страниц существует множество пакетов. Проблема с HTML в том, что в вебе много HTML-страниц, написанных не по стандарту HTML. Впрочем, обработка даже не вполне корректного HTML-кода не так сложна, если под рукой есть подходящие инструменты.

Пакет **Beautiful Soup 4** входит в стандартную поставку Anaconda, но если вы используете другой дистрибутив Python, возможно, вам придётся его установить вручную с помощью pip install beautifulsoup4.

In [0]:
from bs4 import BeautifulSoup

---
Чтобы использовать Beautiful Soup, нужно передать функции BeautifulSoup текст веб-страницы (в виде одной строки). 

html.parser - парсер (программа, которая осуществляет обработку HTML), входит в поставку Python и не требует установки

---


In [0]:
page = BeautifulSoup(req.text, 'html.parser')

In [0]:
page

---
Объект **page** очень похож на строку, на самом деле, это не просто строка. К page можно делать запросы. Например:

---

In [0]:
page.html

---
 Можно пойти вглубь и посмотреть на содержимое `<head>`

---

In [0]:
page.html.head

<head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charset="utf-8"/><title>Apple Inc. (AAPL) Options Chain - Yahoo Finance</title><meta content="AAPL, Apple Inc., AAPL options, Apple Inc. options, AAPL option chain, Apple Inc. option chain, option, option chain, AAPL stock chart, Apple Inc. stock chart, stock chart, stocks, quotes, finance" name="keywords"/><meta content="on" http-equiv="x-dns-prefetch-control"/><meta content="on" property="twitter:dnt"/><meta content="90376669494" property="fb:app_id"/><meta content="#400090" name="theme-color"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="View the basic AAPL option chain and compare options of Apple Inc. on Yahoo Finance." lang="en-US" name="description"/><meta content="guce.yahoo.com" name="oath:guce:consent-host"/><meta content="A9862C0E6E1BE95BCE0BF3D0298FD58B" name="msvalidate.01"/><link href="/man

---
Теперь мы видим только то, что внутри тега `<head>`. Мы можем пойти еще глубже, и получить то, что находится внутри тега `<title>`, который в свою очередь находится внутри тега `<head>` (говорят, что `<title>` является *потомком* `<head>`:

---

In [0]:
page.html.head.title

<title>Apple Inc. (AAPL) Options Chain - Yahoo Finance</title>

In [0]:
page.title

<title>Apple Inc. (AAPL) Options Chain - Yahoo Finance</title>

---
У тегов, кроме названия, бывают еще свойства — например, в строчке `<html lang="en-US">` мы видим свойство `lang` у тега `<html>`, имеющее значение `"en-US"`. 

---

In [0]:
page.html['lang']

'en-US'

---
Другим важным примером тега со свойствами является тег `<a>`, который создает ссылку. У него есть свойство `href`, которое хранит собственно ссылку.


---

In [0]:
page('a')

In [0]:
for link in page('a'):
    print(link['href'])

---
Одним из потомков `<body>` является `<table>`. Ее можно получить вот так.

---

In [0]:
page.body.table

<table class="calls W(100%) Pos(r) Bd(0) Pt(0) list-options" data-reactid="23"><thead data-reactid="24"><tr class="C($c-fuji-grey-j)" data-reactid="25"><th class="Va(b) Py(4px) Fw(400) Fz(xs) Ta(start) Pstart(10px) C($tertiaryColor)" data-reactid="26"><span data-reactid="27">Contract Name</span><!-- react-text: 28 --><!-- /react-text --></th><th class="Va(b) Py(4px) Fw(400) Fz(xs) Ta(end) Pstart(7px) C($tertiaryColor)" data-reactid="29"><span data-reactid="30">Last Trade Date</span><!-- react-text: 31 --><!-- /react-text --></th><th class="Va(b) Py(4px) Fw(400) Fz(xs) Ta(end) Pstart(7px) C($tertiaryColor) C($primaryColor) Fw(500)!" data-reactid="32"><span data-reactid="33">Strike</span><svg class="Va(m)! W(14px) H(14px) Fill($primaryColor)! Stk($primaryColor)! Pt(1px) Pos(a) Cur(p)" data-icon="caret-up" data-reactid="34" height="48" style="fill:#000;stroke:#000;stroke-width:0;vertical-align:bottom;" viewbox="0 0 48 48" width="48"><path d="M24.21 16.03L11.48 28.76c-.78.78-.78 2.047 0 2.

---
Допустим, что нам необходимо получить несколько элементов с одинаковым тегом, например, все строки `<tr>`. Для этого может быть использован такой синтаксис:

---

In [0]:
rows = page.body.table.findAll('tr')
rows

In [0]:
len(rows)

63

---
Мы видим, что список не пуст. Так что по нему можно пройти циклом.

---

In [0]:
for i, row in enumerate(rows):
    print(i)
    print(row)

---
У нас есть строчки и каждая из них является таким же объектом BeautifulSoup, как и все предыдущие. Так что к ним можно применить конструкцию row.td

---

In [0]:
for i, row in enumerate(rows):
    print(i)
    print(row.td)

---
Мы видим, что если внутри тега `<row>` есть несколько тегов `<td>`, то row.td возьмет первый из них. Поэтому мы получили первый столбец. Но нас интересует не сам тег `<td>`, а строка, которая там лежит. Её можно напечатать вот так.

---

In [0]:
del rows[0] # предварительно удалим первый элемент, т.к. он пуст
for i, row in enumerate(rows):
    print(i)
    print(row.td.string)

---
Давайте загрузим таблицу в виде списка списков

---

In [0]:
table = []
for i, row in enumerate(rows):
    table.append([])
    for cell in row.findAll('td'):
        table[-1].append(cell.string)
print(table)

---
Можно написать короче:

---

In [0]:
table = [[cell.string.strip() for cell in row('td')] 
         for row in rows]
print(table)

[['AAPL191220C00100000', '2019-12-04 10:40AM EST', '100.00', '173.10', '173.15', '176.95', '+3.45', '+2.03%', '4', '62', '432.42%'], ['AAPL191220C00105000', '2019-12-09 12:24PM EST', '105.00', '168.10', '168.15', '171.95', '+3.45', '+2.10%', '6', '1', '413.38%'], ['AAPL191220C00110000', '2019-12-12 3:38PM EST', '110.00', '163.85', '163.15', '166.95', '+2.65', '+1.64%', '2', '3', '395.31%'], ['AAPL191220C00115000', '2019-11-06 3:15PM EST', '115.00', '141.62', '153.40', '157.85', '0.00', '-', '3', '1', '0.00%'], ['AAPL191220C00120000', '2019-11-07 9:46AM EST', '120.00', '138.60', '148.50', '152.85', '0.00', '-', '13', '13', '0.00%'], ['AAPL191220C00125000', '2019-11-21 11:46AM EST', '125.00', '137.19', '148.15', '151.90', '0.00', '-', '1', '1', '343.65%'], ['AAPL191220C00130000', '2019-11-06 3:25PM EST', '130.00', '126.17', '138.50', '142.90', '0.00', '-', '10', '8', '0.00%'], ['AAPL191220C00135000', '2019-12-06 11:34AM EST', '135.00', '135.00', '138.15', '141.90', '0.00', '-', '6', '18'

In [0]:
import pandas as pd
dt = pd.DataFrame(table)
dt

### Поиск данных на HTML-странице


Заметим, что в Википедии встречаются ссылки двух типов: внутренние (на другие страницы Википедии) и внешние (на другие сайты), причём они различаются по оформлению — у внешних ссылок есть небольшая стрелочка. Например, мы хотим выбрать все внешние ссылки. Как это сделать?


Для того, чтобы браузер отображал внешние ссылки не так, как внутренние, разработчики Википедии используют так называемые css-классы (конечно, это касается не только Википедии — это вообще основной инструмент современного веба). Теги `<a>`, соответствующие внешним ссылкам, имеют специальный атрибут `class`, значение которого включает слово `external`. Именно по нему можно понять, что речь идёт о внешней ссылке. Это можно было бы увидеть, изучив исходный код страницы, но можно сделать проще: воспользоваться встроенным в браузер инспектором кода.


В исходном коде в атрибуте class тега `<a>` указана строчка "external text", а не просто "external" — дело в том, что теги могут иметь сразу несколько классов одновременно, и в данном случае external и text — это два класса данной ссылки. Но мы будем ориентироваться только на external.

Итак, мы хотим найти все ссылки с классом external. Это очень просто.

In [0]:
from bs4 import BeautifulSoup
import requests

In [0]:
url = "https://ru.wikipedia.org/w/index.php?oldid=75475510"

In [0]:
req = requests.get(url)

In [0]:
req.ok

True

In [0]:
page = BeautifulSoup(req.text, "html.parser")

In [0]:
for link in page.findAll("a", class_='external'):
    print(link['href'])

Как видно из примера выше, достаточно методу `findAll()` передать дополнительный именованный параметр `class_` — обратите внимание на нижнее подчёркивание, без него получится синтаксическая ошибка, потому что слово `class` имеет специальный смысл в Python.

### Классы и поиск по дереву
Решим теперь другую задачу: допустим, мы хотим найти все ссылки в разделе «Примечания», где находятся сноски к основному тексту. С помощью инспектора кода в Firefox (или аналогичных инструментов для других браузеров) мы легко можем заметить, что весь этот раздел находится внутри тега `<div>` (этот тег описывает прямоугольные блоки, из которых состоят веб-страницы, и является основным тегом для современной веб-вёрстки), имеющем класс `reflist columns reflist-narrow`.

In [0]:
divs = page.findAll('div', class_='reflist columns reflist-narrow')

In [0]:
len(divs)

1

---
Такой `<div>` оказался единственным на странице. Найдём теперь все теги `<a>`, являющиеся потомками (возможно, отдалёнными) этого `<div>`'а.

---

In [0]:
div = page.findAll('div', class_='reflist columns reflist-narrow')[0]
for link in div("a")[0:20]:
    print(link['href'])

#cite_ref-_26cc6ad5c5a3baa3_1-0
#CITEREFЛесскис1999
#cite_ref-_6e637976a425c02e_2-0
#cite_ref-_6e637976a425c02e_2-1
#CITEREFЛосев1993
#cite_ref-_26cc6ad5c5a3baa4_3-0
#CITEREFЛесскис1999
#cite_ref-_a0952ae4a2ede05b_4-0
#CITEREFЧудакова1988
#cite_ref-_a0952ae4a2ede05a_5-0
#CITEREFЧудакова1988
#cite_ref-_5f91aa316898efa5_6-0
#cite_ref-_5f91aa316898efa5_6-1
#cite_ref-_5f91aa316898efa5_6-2
#CITEREFСоколов1997
#cite_ref-_a0952be4a2ede229_7-0
#CITEREFЧудакова1988
#cite_ref-8
/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%98%D1%81%D1%82%D0%BE%D1%87%D0%BD%D0%B8%D0%BA%D0%B8_%D0%BA%D0%BD%D0%B8%D0%B3/9785986970592
#cite_ref-9


---
Для экономии места вывеведены только первые 20 ссылок. Это внутренние ссылки на другие фрагменты страницы, поэтому они начинаются с символа `#`.

---

**Подведём некоторые итоги по поводу поиска информации в HTML-файлах:**

- Это всегда творческий процесс: все сайты разные и нет единого рецепта, как извлекать из них нужную информацию.
- В первую очередь нужно посмотреть в исходник интересующей вас странички. Проще всего это делать с помощью инструментария веб-разработчика типа Firebug или встроенного инспектора кода в Firefox или аналогичных инструментов для других браузеров.
- В HTML-дереве можно ориентироваться по названиям тегов, их классам, id'ам и другим свойствам.
- Можно искать нужный элемент итеративно — сначала найти «большой» тег, который включает наш элемент, потом найти в нём элемент поменьше и т.д.

### API и XML

Анализируя веб-страницы и извлекая из них информацию мы пытаемся написать программу, которая бы действовала как человек. Это бывает непросто. К счастью, всё чаще разнообразные сайты предлагают информацию, которую может легко обрабатывать не только человек, но и другая программа. Это называется API — application program interface. Обычный интерфейс — это способ взаимодействия человека с программой, а API — одной программы с другой. Например, вашего скрипта на Python с удалённым веб-сервером.

Для хранения веб-страниц, которые читают люди, используется язык HTML. Для хранения произвольных структурированных данных, которыми обмениваются между собой программы, используются другие языки — в частности, язык XML, похожий на HTML. Вернее было бы сказать, что XML это метаязык, то есть способ описания языков. В отличие от HTML, набор тегов в XML-документе может быть произвольным (и определяется разработчиком конкретного диалекта XML). Например, если бы мы хотели описать в виде XML некоторую студенческую группу, это могло бы выглядеть так:

In [0]:
group = """<group>
<number>134</number>
<student>
<firstname>Виталий</firstname>
<lastname>Иванов</lastname>
</student>
<student>
<firstname>Мария</firstname>
<lastname>Петрова</lastname>
</student>
</group>"""

In [0]:
obj = BeautifulSoup(group, features="xml")
print(obj.prettify())

Вот так мы можем найти в нашем XML-документе номер группы:

In [0]:
obj.group.number.string

'134'

Это значит «в объекте `obj` найти тег `group` в нём найти тег `number` и выдать в виде строки то, что в нём содержится.

А вот так можно перечислить всех студентов:

In [0]:
for student in obj.group.findAll('student'):
    print(student.lastname.string, student.firstname.string)

Иванов Виталий
Петрова Мария




**Задача:** получить списоквсех статей из некоторой категории в Википедии

Вот такой запрос мы можем отправить:

https://en.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=Category:Physics&cmsort=timestamp&cmdir=desc&format=xmlfm

Строка `https://en.wikipedia.org/w/api.php` (до знака вопроса) — это *точка входа* в API. Всё, что идёт после знака вопроса — это, собственно, запрос. Он представляет собой что-то вроде словаря и состоит из пар «ключ=значение», разделяемых амперсандом `&`. Некоторые символы приходится кодировать специальным образом.

Например, в адресе выше сказано, что мы хотим сделать запрос (`action=query`), перечислить элементы категории `list=categorymembers`, в качестве категории, которая нас интересует, указана `Category:Physics` (`cmtitle=Category:Physics`) и указаны некоторые другие параметры. Если кликнуть по этой ссылке, откроется примерно такая штука:

```xml
<?xml version="1.0"?>
<api batchcomplete="">
  <continue cmcontinue="2015-05-30 19:37:50|1653925" continue="-||" />
  <query>
    <categorymembers>
      <cm pageid="24293838" ns="0" title="Wigner rotation" />
      <cm pageid="48583145" ns="0" title="Northwest Nuclear Consortium" />
      <cm pageid="48407923" ns="0" title="Hume Feldman" />
      <cm pageid="48249441" ns="0" title="Phase Stretch Transform" />
      <cm pageid="47723069" ns="0" title="Epicatalysis" />
      <cm pageid="2237966" ns="14" title="Category:Surface science" />
      <cm pageid="2143601" ns="14" title="Category:Interaction" />
      <cm pageid="10844347" ns="14" title="Category:Physical systems" />
      <cm pageid="18726608" ns="14" title="Category:Physical quantities" />
      <cm pageid="22688097" ns="0" title="Branches of physics" />
    </categorymembers>
  </query>
</api>
```



Мы видим здесь разные теги, и видим, что нас интересуют теги `<cm>`, находящиеся внутри тега `<categorymembers>`.

Давайте сделаем соответствующий запрос с помощью Python. Для этого нам понадобится уже знакомый модуль `requests`.

In [0]:
url = "https://en.wikipedia.org/w/api.php"
params = {
    'action':'query',
    'list':'categorymembers',
    'cmtitle': 'Category:Physics',
    'format': 'xml'
}

req = requests.get(url, params=params)
req.ok

True

In [0]:
data = BeautifulSoup(req.text, features='xml')

In [0]:
print(data.prettify())

<?xml version="1.0" encoding="utf-8"?>
<api batchcomplete="">
 <continue cmcontinue="page|2d454147514f294f394543293f042943294f454159011901dc18|48520204" continue="-||"/>
 <query>
  <categorymembers>
   <cm ns="0" pageid="22939" title="Physics"/>
   <cm ns="0" pageid="24489" title="Outline of physics"/>
   <cm ns="100" pageid="1653925" title="Portal:Physics"/>
   <cm ns="0" pageid="5435566" title="Action-angle coordinates"/>
   <cm ns="0" pageid="52657328" title="Bayesian model of computational anatomy"/>
   <cm ns="0" pageid="49342572" title="Group actions in computational anatomy"/>
   <cm ns="0" pageid="61475631" title="Biorheology (journal)"/>
   <cm ns="0" pageid="55503653" title="Camelback potential"/>
   <cm ns="0" pageid="2664158" title="Center of percussion"/>
   <cm ns="0" pageid="62446002" title="Chemical gardening"/>
  </categorymembers>
 </query>
</api>


Найдём все вхождения тега `<cm>` и выведем их атрибут `title`:

In [0]:
for cm in data.api.query.categorymembers("cm"):
    print(cm['title'])

Physics
Outline of physics
Portal:Physics
Action-angle coordinates
Bayesian model of computational anatomy
Group actions in computational anatomy
Biorheology (journal)
Camelback potential
Center of percussion
Chemical gardening


Можно было упростить поиск `<cm>`, не указывая «полный путь» к ним:

In [0]:
for cm in data("cm"):
    print(cm['title'])

Physics
Outline of physics
Portal:Physics
Action-angle coordinates
Bayesian model of computational anatomy
Group actions in computational anatomy
Biorheology (journal)
Camelback potential
Center of percussion
Chemical gardening


По умолчанию сервер вернул нам список из 10 элементов. Если мы хотим больше, нужно воспользоваться элементом continue — это своего рода гиперссылка на следующие 10 элементов.

In [0]:
data.find("continue")['cmcontinue']

'page|2d454147514f294f394543293f042943294f454159011901dc18|48520204'

Необходимо использовать метод find() вместо того, чтобы просто написать data.continue, потому что continue в Python имеет специальный смысл.

Теперь добавим cmcontinue в наш запрос и выполним его ещё раз:

In [0]:
params['cmcontinue'] = data.api("continue")[0]['cmcontinue']

In [0]:
req = requests.get(url, params=params)
data = BeautifulSoup(g.text, features='xml')
for cm in data.api.query.categorymembers("cm"):
    print(cm['title'])

Мы получили следующие 10 элементов из категории. Продолжая таким образом, можно выкачать её даже целиком (правда, для этого потребуется много времени).

Аналогичным образом реализована работа с разнообразными другими API, имеющимися на разных сайтах. Где-то API является полностью открытым (как в Википедии), где-то вам потребуется зарегистрироваться и получить application id и какой-нибудь ключ для доступа к API, где-то попросят даже заплатить (например, автоматический поиск в Google). Есть API, которые позволяют только читать информацию, а бывают и такие, которые позволяют её править. Например, можно написать скрипт, который будет автоматически сохранять какую-то информацию в Google Spreadsheets. Всякий раз при использовании API вам придётся изучить его документацию, но это в любом случае проще, чем обрабатывать HTML-код. Иногда удаётся упростить доступ к API, используя специальные библиотеки.

## Задание

1.	Необходимо получить данные (название, ссылка, стоимость) с https://market.yandex.ru/ о какой-нибудь группе товаров. Например: https://market.yandex.ru/search?text=Конструкторы%20LEGO&cvredirect=0&track=redirbarup&local-offers-first=0
2.	Представить собранные данные в виде DataFrame, записать в файл.
3.	Кратко пояснить выбор инструментов сбора данных (HTML, API (xml/json)). 

P.S.: Можно выбрать другой ресурс для сбора данных и/или изменить список полей итогового DataFrame по своему усмотрению. Аргументировать/согласовать выбор источника/набор данных.


Я решил взять ресурс [kinopoisk.ru](https://kinopoisk.ru). Сначала я хотел использовать магазины бытовой техники, но на большинстве стоит защита от парсинга, поэтому я и решил сменить предметную область.

С кинопоиска мы берем список премьер и забираем их название, ссылку на страницу фильма, год выпуска и компанию-производитель.

Поскольку получить интересующую нас информацию из html-коды было несложно, было принято решение использовать requests+BeautifulSoup

In [176]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

r = requests.get('https://www.kinopoisk.ru/premiere/')
page = BeautifulSoup(r.text, "html.parser")
films = page.findAll('div', class_='premier_item')
titles = [item.find("div",class_='textBlock').find('a').text for item in films]
urls = ['https://www.kinopoisk.ru' + item.find("div",class_='textBlock').find('a')['href'] for item in films]
years = [item.find("div",class_='textBlock').findAll('span')[1].text.split('(')[1].split(')')[0] for item in films]
companies = [item.find("s",class_='company').find('a').text for item in films]
dt = pd.DataFrame({'Title':titles,'URL':urls,'Relise year':years,'Company':companies})
dt.to_csv("premieres.csv")
dt

,Title,URL,Relise year,Company
0,Джуманджи: Новый уровень,https://www.kinopoisk.ru/film/1112539/,2019,WDSSPR
1,Полицейский с Рублевки. Новогодний беспредел 2,https://www.kinopoisk.ru/film/1255106/,2019,Централ Партнершип
2,Курьер,https://www.kinopoisk.ru/film/1207545/,2019,Вольга
3,Озеро диких гусей,https://www.kinopoisk.ru/film/1236072/,2019,Престиж Кино
4,Одной волшебной ночью,https://www.kinopoisk.ru/film/1253425/,2019,ПРОвзгляд
5,Тайна Мосли,https://www.kinopoisk.ru/film/1034049/,2019,MEGOGO Distribution
6,Собаки не носят штанов,https://www.kinopoisk.ru/film/1200332/,2019,HHG
7,Царь зверей,https://www.kinopoisk.ru/film/1203112/,2018,Ракета Релизинг
8,Щенячий патруль: Мегащенки и отважные птенцы,https://www.kinopoisk.ru/film/1328006/,2019,Пионер
9,GORILLAZ: Долой фальшивых идолов,https://www.kinopoisk.ru/film/1325555/,2019,CoolConnections
